In [1]:
import pymc as pm
import pandas as pd
import numpy as np

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:

df = pd.read_csv('../data/data-tidy/perc_disagreement_in_time.csv')

In [3]:
df = df[df.submission_id=='12a7mks']
df.reset_index(drop=True, inplace=True)
df['created'] = pd.to_datetime(df['created'], format='%Y-%m-%d %H:%M:%S')
start = df.iloc[0]['created']
eighteen_h = start + pd.Timedelta(18, 'h', hours=18)

# set the streshold (18h) and split the dataframe
df_before = df[df['created'] < eighteen_h]
df_after = df[df['created'] > eighteen_h]
df_before.reset_index(drop=True, inplace=True)
df_after.reset_index(drop=True, inplace=True)

In [4]:
Bh_before = df_before[df_before.created > df_before.iloc[-1]['created']-pd.Timedelta(10, 'h')]
Bh_after = df_after[df_after.created < df_after.iloc[0]['created']+pd.Timedelta(10, 'h')]

#Bh_before['final_verdict'] = 0
#Bh_after['final_verdict'] = 1

Bh_before = Bh_before.iloc[0][['ESH_perc', 'NAH_perc', 'NTA_perc', 'YTA_perc', 'unsure_perc', 'no_vote_perc']].to_dict()
Bh_after = Bh_after.iloc[-1][['ESH_perc', 'NAH_perc', 'NTA_perc', 'YTA_perc', 'unsure_perc', 'no_vote_perc']].to_dict()
print(Bh_before, '\n', Bh_after)

{'ESH_perc': 0.0144508670520231, 'NAH_perc': 0.0239471511147811, 'NTA_perc': 0.2303881090008257, 'YTA_perc': 0.1123038810900082, 'unsure_perc': 0.0103220478943022, 'no_vote_perc': 0.6085879438480595} 
 {'ESH_perc': 0.0148725212464589, 'NAH_perc': 0.0179886685552407, 'NTA_perc': 0.2082152974504249, 'YTA_perc': 0.1028328611898017, 'unsure_perc': 0.0109065155807365, 'no_vote_perc': 0.6451841359773371}


$$ y \sim \text{Normal}(\mu, \sigma) $$
$$ \mu = \alpha + \beta_1 B_h + \beta_2 T $$
$$ \alpha \sim \text{Normal}(\mu_\alpha, \sigma_\alpha) $$
$$ \beta_1 \sim \text{Normal}(\mu_{\beta_1}, \sigma_{\beta1}) $$
$$ \beta_2 \sim \text{Normal}(\mu_{\beta_2}, \sigma_{\beta_2}) $$
$$ \sigma \sim \text{Uniform}(0, s) $$


In [5]:
Bh = np.array(list(Bh_before.values()))
print(Bh)

y = np.array(list(Bh_after.values()))
print(y)

#verdict = 

[0.01445087 0.02394715 0.23038811 0.11230388 0.01032205 0.60858794]
[0.01487252 0.01798867 0.2082153  0.10283286 0.01090652 0.64518414]


In [6]:
# Bh = Bh.reshape(-1, 1)

In [7]:
with pm.Model() as m_SW:

    # priors
    alpha = pm.Normal("alpha", 50, 10) 
    sigma = pm.Uniform("sigma", 0, 1)

    #t = alpha[verdict]
    beta1 = pm.Normal("beta1", 0, 1)
    #beta2 = pm.Normal("beta2", 0, 1)  

    mu = alpha + beta1*Bh #+ beta2*T  # linear model # deterministic variable
    Ylikelihood  = pm.Normal("y", mu, sigma, observed=y)  # likelihood

    # posterior
    idata_m_SHW = pm.sample(draws=1000, tune=1000, return_inferencedata=True, progressbar=True)


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


In [ ]:
print(Bh)